# Progetto CS

Paolo Merola & Mattia Fratello

In [1]:
# Starting from the The Movie DataBase (TMDB) sample datasets:

import pandas as pd
import numpy as np

cr = pd.read_csv("C:/Users/pablo/Desktop/Computer Science (Python)/Progetto/tmdb_5000_credits.csv")
mv = pd.read_csv("C:/Users/pablo/Desktop/Computer Science (Python)/Progetto/tmdb_5000_movies.csv")
mv.rename(columns = {'id': 'movie_id'},inplace = True)
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
def check_uniqueness(df, column):
    return len(df[column]) == len(df[column].drop_duplicates())
check_uniqueness(cr,'movie_id') and check_uniqueness(mv,'movie_id')

True

In [2]:
#FUNZIONE PER ESTRARRE I DATAFRAME SCRITTI IN JSON
def extract_json(df, column):
    result = df[column].apply(pd.read_json) #result è una Series di Dataframe
    movie_id = df['movie_id'] #Series degli ID dei film, nello stesso ordine dei dataframe contenuti in result
    for i in range(len(df)):
        result[i]['movie_id'] = movie_id[i] #attacco ad ogni dataframe l'id del film a cui si riferisce
    return pd.concat(result.values).reset_index().drop("index", axis = 1) #dalla Series di Dataframe ricavo un unico Dataframe

In [3]:
#ESTRAGGO TUTTI I JSON ED ELIMINO LE CORRISPONDENTI COLONNE NEI DATAFRAME INIZIALI
cast = extract_json(cr,'cast')

crew = extract_json(cr,'crew')

genres = extract_json(mv,'genres')

keywords = extract_json(mv,'keywords')

production_companies = extract_json(mv,'production_companies')

production_countries = extract_json(mv,'production_countries')

spoken_languages = extract_json(mv,'spoken_languages')

movies = mv.drop(['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages'],axis = 1)
credits = cr.drop(['cast', 'crew'], axis = 1)

RISOLUZIONE DELLE QUERY SULLE TABELLE NON NORMALIZZATE

In [4]:
#Rendo più leggibili cast e crew: rinomino le colonne con nomi parlanti, setto indici appropriati, 
# trasformo il gender da valore numerico a stringa e rimetto ad interi i valori castati a float per errore
cast.rename(columns = {'id': 'actor_id'}, inplace = True)
crew.rename(columns = {'id': 'crew_id'}, inplace = True)

cast.set_index('credit_id', inplace=True)
crew.set_index('credit_id',inplace=True)

cast.replace({"gender": {0.0: np.nan, 1.0: "F", 2.0: "M"}}, inplace = True)
crew.replace({"gender": {0.0: np.nan, 1.0: "F", 2.0: "M"}}, inplace = True)

cast[['cast_id', 'actor_id', 'order']] = cast[['cast_id','actor_id', 'order']].applymap(int)
crew['crew_id'] = crew['crew_id'].apply(int)

In [5]:
cast.isnull().values.any()

True

In [6]:
# 1. For each movie, compute the number of cast members

actors_each_movie = cast.groupby(by = ['movie_id'], as_index = True).size().to_frame().rename(
    columns = {0:'number of actors'})
#actors_each_movie

In [7]:
# 2. How many movies do not have a homepage?
num_movies_no_homepage = len(mv[mv['homepage'].isnull()])
print(num_movies_no_homepage,'movies do not have a homepage')

3091 movies do not have a homepage


In [8]:
#3. For each year, how many movies do not have a homepage?
def extract_year(timestamp):
    if timestamp is np.nan:
        return np.nan
    else:
        return pd.to_datetime(timestamp).year

mv2 = mv[['homepage','release_date']].dropna(subset = ['release_date'])
mv2['release_year'] = mv2['release_date'].apply(extract_year) 
movies_per_year = mv2[mv2['homepage'].isnull()].groupby(by='release_year').size().to_frame().rename(columns = {0 : 'number_of_movies'})

In [9]:
#4. Extract the domain of each homepage (for example .com, .it, .org)

def extract_domain(hp):
    domains = []
    if hp is np.nan:
        return np.nan
    else:
        homepage = hp.split(' ')
        for i in range(len(homepage)):
            domains.append(homepage[i].split('/')[2].split('.')[-1])
        return str(domains)[1:-1].replace("\'", "")
    
domains = mv['homepage'].apply(extract_domain).to_frame()
domains_with_titles = mv['movie_id'].to_frame().join(domains).rename(columns = {'homepage':'domain'}).set_index('movie_id')

In [10]:
#6. For each movie, compute the gross margin (difference between revenue and budget)
mv['gross_margin'] =  mv['revenue'] - mv['budget']
gross_margin_movie = mv[['movie_id','gross_margin']].set_index('movie_id')
#gross_margin_movie

In [11]:
#7. For each movie, compute the number of crew members
num_crew_each_movie = crew.groupby(by = 'movie_id').size().to_frame().rename(columns = {0: 'num_crew_members'})
#num_crew_each_movie

In [12]:
#8. For each movie, compute the number of directors
num_crew_directors_movie = crew[crew['job']=='Director'].groupby(by = 'movie_id',as_index = True).size().to_frame().rename(
columns = {0: 'num_directors'})
#num_crew_directors_movie

In [13]:
#9. For each language, compute the number of movies where such language is spoken.
num_movies_each_language = spoken_languages.groupby(by = 'iso_639_1',as_index=True).size().to_frame().rename(columns = {0:'Number of movies where such language is spoken'})
#num_movies_each_language

In [14]:
#rendo più leggibile production_companies

production_companies.rename(columns = {'id': 'company_id'}, inplace = True)
production_companies['company_id'] = production_companies['company_id'].apply(int)

In [15]:
#10. For each company and each decade, compute the overall revenue
def extract_decade(timestamp):
    return str(extract_year(timestamp))[:3] + '0'

mv['release_decade'] = mv['release_date'].apply(extract_decade)

mv_companies = production_companies.merge(right = mv[['movie_id', 'revenue','release_decade']], on = 'movie_id')
revenue_company_decade = mv_companies[['company_id','revenue','release_decade']].groupby(by = ['company_id', 'release_decade']).sum()

In [16]:
# 11. For each decade, compute the company with maximum revenue

def extract_first_from_tuple(tup):
    return tup[0]
max_revenue_per_company_per_decade = revenue_company_decade.groupby(level=1).idxmax().rename(columns = {'revenue':'most_solvable_company'})
max_revenue_per_company_per_decade['most_solvable_company'] = max_revenue_per_company_per_decade['most_solvable_company'].apply(extract_first_from_tuple)

In [17]:
#12. In each year, how many movies have revenue smaller than the budget?

mv3 = mv[['release_date','gross_margin']].dropna(subset = ['release_date'])
mv3['release_year'] = mv3['release_date'].apply(extract_year) 
num_failure_movies_year = mv3[mv3['gross_margin'] < 0].groupby(by = 'release_year',as_index=True).size()

In [198]:
cast.head()

,cast_id,character,gender,actor_id,movie_id,name,order
credit_id,,,,,,,
5602a8a7c3a3685532001c9a,242,Jake Sully,M,65731,19995,Sam Worthington,0
52fe48009251416c750ac9cb,3,Neytiri,F,8691,19995,Zoe Saldana,1
52fe48009251416c750aca39,25,Dr. Grace Augustine,F,10205,19995,Sigourney Weaver,2
52fe48009251416c750ac9cf,4,Col. Quaritch,M,32747,19995,Stephen Lang,3
52fe48009251416c750ac9d3,5,Trudy Chacon,F,17647,19995,Michelle Rodriguez,4


In [196]:
#13. Distribute the revenue according to the order of appearance in a movie. 
#    Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

#legge di distribuzione: 2^(n-k-1)/(2^n -1) * revenue
#TODO (numeri troppo grandi)
def actor_part(orders,num_actors,revenues):
    ar = np.array([0]*len(orders),dtype=object)
    for i in range(len(orders)):
        n = num_actors[i]
        k = orders[i]
        rev = revenues[i]
        
        fac1 = (2**n)/(2**n - 1)
        fac2 = 1/(2**(k+1))
       
        number = fac1 * fac2 * rev
        print('n= ', n, 'k= ',k, 'rev=', rev,'\ni= ',i,' number: ', number, 'ar: ', ar)
        ar[i] = number
    return ar
   

In [167]:
movies_revenue = mv_companies[['movie_id','revenue']].drop_duplicates()
cast2=pd.DataFrame()
cast2 = cast.merge(right = actors_each_movie.reset_index(), on = 'movie_id').merge(right = movies_revenue, on = 'movie_id')
cast2 = cast2[['actor_id','movie_id','order','number of actors','revenue']]
cast2.head()

,actor_id,movie_id,order,number of actors,revenue
0,65731,19995,0,83,2787965087
1,8691,19995,1,83,2787965087
2,10205,19995,2,83,2787965087
3,32747,19995,3,83,2787965087
4,17647,19995,4,83,2787965087


In [200]:
cast3 = cast2[cast2['number of actors']<10]
cast3.head(1)

,actor_id,movie_id,order,number of actors,revenue
7518,55636,2114,0,9,85131830


In [202]:
actor_part(cast2['order'].values,cast2['number of actors'].values,cast2['revenue'].values)


103104
n=  83 k=  0 rev= 2787965087 
i=  0  number:  -0.0 ar:  [0 0 0 ..., 0 0 0]
n=  83 k=  1 rev= 2787965087 
i=  1  number:  -0.0 ar:  [-0.0 0 0 ..., 0 0 0]
n=  83 k=  2 rev= 2787965087 
i=  2  number:  -0.0 ar:  [-0.0 -0.0 0 ..., 0 0 0]
n=  83 k=  3 rev= 2787965087 
i=  3  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  4 rev= 2787965087 
i=  4  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  5 rev= 2787965087 
i=  5  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  6 rev= 2787965087 
i=  6  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  7 rev= 2787965087 
i=  7  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  8 rev= 2787965087 
i=  8  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  9 rev= 2787965087 
i=  9  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  10 rev= 2787965087 
i=  10  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  83 k=  11 rev= 2787965087 
i=  11  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]

C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in longlong_scalars
  
C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


 [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  18 rev= 961000000 
i=  101  number:  1832.96203624 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  19 rev= 961000000 
i=  102  number:  916.48101812 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  20 rev= 961000000 
i=  103  number:  458.24050906 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  21 rev= 961000000 
i=  104  number:  229.12025453 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  22 rev= 961000000 
i=  105  number:  114.560127265 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  23 rev= 961000000 
i=  106  number:  57.2800636325 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  24 rev= 961000000 
i=  107  number:  28.6400318162 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  25 rev= 961000000 
i=  108  number:  14.3200159081 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  26 rev= 961000000 
i=  109  number:  7.16000795406 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  27 rev= 961000000 
i=  110  number:  3.58000397703 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  34 k=  28 rev= 96100

n=  158 k=  99 rev= 1084939099 
i=  298  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  100 rev= 1084939099 
i=  299  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  101 rev= 1084939099 
i=  300  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  102 rev= 1084939099 
i=  301  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  103 rev= 1084939099 
i=  302  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  104 rev= 1084939099 
i=  303  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  105 rev= 1084939099 
i=  304  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  106 rev= 1084939099 
i=  305  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  107 rev= 1084939099 
i=  306  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  108 rev= 1084939099 
i=  307  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  109 rev= 1084939099 
i=  308  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  158 k=  110 rev= 1084939099 


n=  143 k=  117 rev= 890871626 
i=  502  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  118 rev= 890871626 
i=  503  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  119 rev= 890871626 
i=  504  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  120 rev= 890871626 
i=  505  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  121 rev= 890871626 
i=  506  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  122 rev= 890871626 
i=  507  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  123 rev= 890871626 
i=  508  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  124 rev= 890871626 
i=  509  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  125 rev= 890871626 
i=  510  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  126 rev= 890871626 
i=  511  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  127 rev= 890871626 
i=  512  number:  nan ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  143 k=  128 rev= 890871626 
i=  513  nu

...

i=  11081  number:  30.9944152832 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  24 rev= 520000000 
i=  11082  number:  15.4972076416 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  25 rev= 520000000 
i=  11083  number:  7.7486038208 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  26 rev= 520000000 
i=  11084  number:  3.8743019104 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  27 rev= 520000000 
i=  11085  number:  1.9371509552 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  28 rev= 520000000 
i=  11086  number:  0.9685754776 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  29 rev= 520000000 
i=  11087  number:  0.4842877388 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  30 rev= 520000000 
i=  11088  number:  0.2421438694 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  31 rev= 520000000 
i=  11089  number:  0.1210719347 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  32 rev= 520000000 
i=  11090  number:  0.06053596735 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  42 k=  33 rev= 520000000 
i=  11091  number:  0.030267983675 ar

n=  110 k=  44 rev= 266465037 
i=  11193  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  45 rev= 266465037 
i=  11194  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  46 rev= 266465037 
i=  11195  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  47 rev= 266465037 
i=  11196  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  48 rev= 266465037 
i=  11197  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  49 rev= 266465037 
i=  11198  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  50 rev= 266465037 
i=  11199  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  51 rev= 266465037 
i=  11200  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  52 rev= 266465037 
i=  11201  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  53 rev= 266465037 
i=  11202  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  54 rev= 266465037 
i=  11203  number:  -0.0 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  110 k=  55 rev= 2

n=  29 k=  28 rev= 170432927 
i=  11337  number:  0.317456065337 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  0 rev= 242295562 
i=  11338  number:  121148243.144 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  1 rev= 242295562 
i=  11339  number:  60574121.5719 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  2 rev= 242295562 
i=  11340  number:  30287060.786 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  3 rev= 242295562 
i=  11341  number:  15143530.393 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  4 rev= 242295562 
i=  11342  number:  7571765.19649 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  5 rev= 242295562 
i=  11343  number:  3785882.59825 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  6 rev= 242295562 
i=  11344  number:  1892941.29912 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  7 rev= 242295562 
i=  11345  number:  946470.649561 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  8 rev= 242295562 
i=  11346  number:  473235.324781 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  18 k=  9 rev= 242295562 
i=  11347  

n=  11 k=  0 rev= 169327687 
i=  11488  number:  84705203.4626 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  2 rev= 169327687 
i=  11489  number:  21176300.8657 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  4 rev= 169327687 
i=  11490  number:  5294075.21641 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  6 rev= 169327687 
i=  11491  number:  1323518.8041 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  8 rev= 169327687 
i=  11492  number:  330879.701026 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  9 rev= 169327687 
i=  11493  number:  165439.850513 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  10 rev= 169327687 
i=  11494  number:  82719.9252565 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  12 rev= 169327687 
i=  11495  number:  20679.9813141 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  13 rev= 169327687 
i=  11496  number:  10339.9906571 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  14 rev= 169327687 
i=  11497  number:  5169.99532853 ar:  [-0.0 -0.0 -0.0 ..., 0 0 0]
n=  11 k=  15 rev= 169327687 
i=  114

KeyboardInterrupt: 

In [ ]:
#14. For each actor find the total revenue attributed to him/her.
actors_total_revenues = mv_actors.groupby(by = 'name').sum()['revenue_part']
actors_total_revenues

In [ ]:
#15. Find the actor that is responsible for the most overall revenue.
actors_total_revenues[actors_total_revenues == actors_total_revenues.max()]

NORMALIZZAZIONE DELLE TABELLE DA QUI IN POI

In [54]:
#Normalizzo cast e crew

actors = cast[['actor_id' , 'name', 'gender']].set_index('actor_id')

workers = crew[['crew_id','name','gender']].set_index('crew_id')

cast.drop(['name', 'gender' ], axis = 1, inplace = True)

crew.drop(['name', 'gender' ], axis = 1, inplace = True)

In [65]:
genres.rename(columns = {'id': 'genre_id'}, inplace = True)
genres['genre_id'] = genres['genre_id'].apply(int)
genres_names = genres[['genre_id','name']].set_index('genre_id').drop_duplicates().sort_index()
genres.drop('name',inplace = True, axis = 1)
genres_names

In [72]:
keywords.rename(columns = {'id': 'keyword_id'}, inplace = True)
keywords['keyword_id'] = keywords['keyword_id'].apply(int)
keywords_names = keywords[['keyword_id','name']].set_index('keyword_id').drop_duplicates().sort_index()
keywords.drop('name',inplace = True, axis = 1)
keywords_names

,name
keyword_id,
30,individual
65,holiday
74,germany
75,gunslinger
83,saving the world
90,paris
100,slum
107,barcelona spain
108,transvestism


In [74]:
production_companies_names = production_companies[['company_id','name']].set_index('company_id').drop_duplicates().sort_index()
production_companies.drop('name',inplace = True, axis = 1)
production_companies_names

,name
company_id,
1,Lucasfilm
2,Walt Disney Pictures
3,Pixar Animation Studios
4,Paramount Pictures
5,Columbia Pictures
6,RKO Radio Pictures
7,DreamWorks
8,Fine Line Features
9,Gaumont


In [76]:
production_countries_names = production_countries[['iso_3166_1','name']].set_index('iso_3166_1').drop_duplicates().sort_index()
production_countries.drop('name',inplace = True, axis = 1)
production_countries_names

,name
iso_3166_1,
AE,United Arab Emirates
AF,Afghanistan
AO,Angola
AR,Argentina
AT,Austria
AU,Australia
AW,Aruba
BA,Bosnia and Herzegovina
BE,Belgium


In [83]:
spoken_languages_names = spoken_languages[['iso_639_1','name']].set_index('iso_639_1').drop_duplicates().sort_index()
spoken_languages.drop('name',inplace = True, axis = 1)
spoken_languages_names


,name
iso_639_1,
af,Afrikaans
ar,العربية
bg,български език
bm,Bamanankan
bn,বাংলা
bo,
bs,Bosanski
ca,Català
cn,广州话 / 廣州話
